In [1]:
import urllib.request
import csv
import glob
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import numpy as np
import os

# Scrape Fanfooty data
This notebook is used to scrape the following data from fanfooty:

1. Current player list
2. Match stats for each player
3. Match results/fixture

## REQUIRED - Specify match IDs to scrape:

In [2]:
start_match = 7423
end_match = 7454

### Create folder to save all outputs

In [3]:
timestr = time.strftime("%Y%m%d-%H%M%S")
destination = "exports/scrape_{}".format(timestr)
os.mkdir(destination)

## 1. Current player list

### Save historical fanfooty player IDs csv file

In [4]:
url = "http://www.fanfooty.com.au/resource/player.php?type=all"
headers = "fanfooty_player_id,fanfooty_afl_player_id,fanfooty_first_name,fanfooty_surname,fanfooty_team,fanfooty_status,fanfooty_number,fanfooty_birth_date,fanfooty_height,fanfooty_weight,fanfooty_state_of_origin,fanfooty_recruited_from,fanfooty_games,fanfooty_goals"

res = requests.get(url)
split_rows_list = res.text.split('"\r\n')
f = open('{}/fanfooty_player_ids_historical.csv'.format(destination),'w')
f.write('{}\n'.format(headers)) 
for row in split_rows_list:
    f.write('{}\n'.format(row)) 
f.close()

### Clean data:
* Remove duplicate players
* Standard team name

In [5]:
df_player_list = pd.read_csv('{}/fanfooty_player_ids_historical.csv'.format(destination), encoding = "ISO-8859-1")
df_team_names = pd.read_csv('inputs/all_sources_team_names.csv')
df_player_list['short_team'] = df_player_list['fanfooty_team'].map(dict(df_team_names[['fanfooty_team_name_mid', 'sc_team_name']].values))
df_player_list = df_player_list.dropna(subset=['fanfooty_afl_player_id'])
df_player_list = df_player_list[df_player_list['fanfooty_afl_player_id'] != 0]

remove_duplicate_players = [
    {'player_id': 271128, 'first_name': 'Danny', 'surname': 'Butcher', 'team': 'Port Adelaide'},
    {'player_id': 296214, 'first_name': 'Jake', 'surname': 'Barratt', 'team': 'GWS'},
    {'player_id': 991933, 'first_name': 'Jason', 'surname': 'Cantstandya', 'team': 'Richmond'},
    {'player_id': 992351, 'first_name': 'Bailey', 'surname': 'Williams', 'team': 'None'}
]

for player in remove_duplicate_players:
    indexNames = df_player_list[
        (df_player_list['fanfooty_afl_player_id'] == player['player_id']) & 
        (df_player_list['fanfooty_first_name'] == player['first_name']) &
        (df_player_list['fanfooty_surname'] == player['surname']) &
        (df_player_list['fanfooty_team'] == player['team'])
    ].index
    df_player_list.drop(indexNames , inplace=True)

df_player_list.to_csv('{}/fanfooty_player_ids_historical.csv'.format(destination))
df_player_list

,fanfooty_player_id,fanfooty_afl_player_id,fanfooty_first_name,fanfooty_surname,fanfooty_team,fanfooty_status,fanfooty_number,fanfooty_birth_date,fanfooty_height,fanfooty_weight,fanfooty_state_of_origin,fanfooty_recruited_from,fanfooty_games,fanfooty_goals,short_team
3,43031,294472.0,Rory,Atkins,Adelaide,senior,21.0,1994-07-12,186.0,85.0,VIC,Calder Cannons,101.0,47.0,ADE
6,1011,980001.0,Nathan,Bassett,Adelaide,NaN,8.0,1976-12-07,190.0,88.0,SA,Norwood,210.0,25.0,ADE
7,44021,295026.0,James,Battersby,Adelaide,NaN,43.0,1995-11-07,177.0,78.0,SA,Sturt,NaN,NaN,ADE
8,46114,280974.0,Jonathon,Beech,Adelaide,NaN,1.0,1990-11-09,187.0,85.0,SA,West Adelaide,3.0,1.0,ADE
9,623,990006.0,James,Begley,Adelaide,NaN,28.0,1980-07-22,0.0,0.0,NaN,NaN,61.0,10.0,ADE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,47084,993894.0,Lachlan,Tiziani,GWS,NaN,33.0,1997-03-13,189.0,79.0,NSW,Broken Hill,0.0,0.0,GWS
5045,42215,291928.0,Gerald,Ugle,GWS,NaN,32.0,1993-01-31,180.0,80.0,WA,Beverley,3.0,1.0,GWS
5046,39670,280109.0,Callan,Ward,GWS,senior,8.0,1990-04-10,187.0,84.0,VIC,Western Jets,223.0,110.0,GWS
5047,43001,294305.0,Lachie,Whitfield,GWS,senior,6.0,1994-07-18,187.0,81.0,VIC,Dandenong Stingrays,151.0,63.0,GWS


## 2. Match stats for each player

### Scrape match files from Fanfooty website

In [6]:
def return_list_of_urls(match_id):
    full_url_list = []
    for match in match_id:
        url = "http://live.fanfooty.com.au/chat/"
        extension = ".txt"
        full_url = "{}{}{}".format(url, match, extension)
        full_url_list.append(full_url)
    return full_url_list



matches = list(range(start_match, end_match + 1))

list_of_urls = return_list_of_urls(matches)

for list in list_of_urls:
    print(list)

for url in list_of_urls:
    response = urllib.request.urlopen(url)
    webContent = response.read()
    filename = url[-8:]
    f = open("inputs/All Match Data/{}".format(filename), 'wb')
    f.write(webContent)
    print(filename)

http://live.fanfooty.com.au/chat/7423.txt
http://live.fanfooty.com.au/chat/7424.txt
http://live.fanfooty.com.au/chat/7425.txt
http://live.fanfooty.com.au/chat/7426.txt
http://live.fanfooty.com.au/chat/7427.txt
http://live.fanfooty.com.au/chat/7428.txt
http://live.fanfooty.com.au/chat/7429.txt
http://live.fanfooty.com.au/chat/7430.txt
http://live.fanfooty.com.au/chat/7431.txt
http://live.fanfooty.com.au/chat/7432.txt
http://live.fanfooty.com.au/chat/7433.txt
http://live.fanfooty.com.au/chat/7434.txt
http://live.fanfooty.com.au/chat/7435.txt
http://live.fanfooty.com.au/chat/7436.txt
http://live.fanfooty.com.au/chat/7437.txt
http://live.fanfooty.com.au/chat/7438.txt
http://live.fanfooty.com.au/chat/7439.txt
http://live.fanfooty.com.au/chat/7440.txt
http://live.fanfooty.com.au/chat/7441.txt
http://live.fanfooty.com.au/chat/7442.txt
http://live.fanfooty.com.au/chat/7443.txt
http://live.fanfooty.com.au/chat/7444.txt
http://live.fanfooty.com.au/chat/7445.txt
http://live.fanfooty.com.au/chat/7

### Headers of each field in match file

In [7]:
column_header_names = [
    'Fanfooty Match ID',
    'Fanfooty Match URL',
    'Round',
    'Year',
    'Player ID',
    'First Name',
    'Surname',
    'Team',
    'null',
    'DT',
    'SC',
    'null2',
    'null3',
    'null4',
    'Kicks',
    'Handballs',
    'Marks',
    'Tackles',
    'Hitouts',
    'Frees for',
    'Frees against',
    'Goals',
    'Behinds',
    'Not sure',
    'Tag',
    'Tag Notes',
    'Tag 2',
    'Tag 2 Notes',
    'null5',
    'null6',
    'null7',
    'null8',
    'Position',
    'Jumper Number',
    'null9',
    'null10',
    'null11',
    'DT own %',
    'SC own %',
    'AF own %',
    'null12',
    'AF Breakeven',
    'null13',
    'Contested Possessions',
    'Clearances',
    'Clangers',
    'Disposal efficiency',
    'Time on ground',
    'Metres gained'
]

### Read match files and write to csv

In [8]:
df_fanfooty_player_raw = pd.DataFrame()
def get_number_of_lines_in_file(data):
    return len(data.split('\n'))


def get_match_id(data):
    name = data.split('\n', 1)[0]
    return name[-8:-4]


def get_url_of_match(data):
    name = data.split('\n', 1)[0]
    url = "http://live.fanfooty.com.au/game/matchcentre.html?id=" + name[-8:-4]
    return url


def get_round(data):
    line = data.split('\n', 1)[1]
    stripped_line = [x.strip() for x in line.split(',')]
    afl_round = stripped_line[4]
    return afl_round


def get_year(data):
    second_line = data.splitlines()[2]
    stripped_second_line = [x.strip() for x in second_line.split(',')]
    afl_year = stripped_second_line[1]
    return afl_year


def get_match_data_list():
    data_list = []
    path = "inputs/All Match Data/*.txt"

    for item in glob.glob(path):
        file = open(item, 'r')
        name = file.name
        data = file.read()
        data_list.append(name + '\n' + data)
    return data_list


def return_player_match_data(data_list):
    player_data_for_match = []

    for match in data_list:
        number_of_lines = get_number_of_lines_in_file(match)
        afl_round = get_round(match)
        afl_year = get_year(match)
        name = get_url_of_match(match)
        match_id = get_match_id(match)

        for line in range(5, number_of_lines - 1):
            line_data = match.splitlines()[line]
            line_data = [x.strip() for x in line_data.split(',')]
            line_data = [match_id] + [name] + [afl_round] + [afl_year] + line_data
            player_data_for_match.append(line_data)
    return player_data_for_match


match_data_list = get_match_data_list()
player_data = return_player_match_data(match_data_list)
file_name = "fanfooty_match_data_{}.csv".format(timestr)
with open("{}/{}".format(destination, file_name), "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(column_header_names)
    for item in player_data:
        writer.writerow(item)

df_fanfooty_player_raw = pd.read_csv("{}/{}".format(destination, file_name), error_bad_lines=False)
df_fanfooty_player_raw

b'Skipping line 89678: expected 49 fields, saw 50\nSkipping line 89679: expected 49 fields, saw 50\nSkipping line 89680: expected 49 fields, saw 50\nSkipping line 89681: expected 49 fields, saw 50\nSkipping line 89682: expected 49 fields, saw 50\nSkipping line 89683: expected 49 fields, saw 50\nSkipping line 89684: expected 49 fields, saw 50\nSkipping line 89685: expected 49 fields, saw 50\nSkipping line 89686: expected 49 fields, saw 50\nSkipping line 89687: expected 49 fields, saw 50\nSkipping line 89688: expected 49 fields, saw 50\nSkipping line 89689: expected 49 fields, saw 50\nSkipping line 89690: expected 49 fields, saw 50\nSkipping line 89691: expected 49 fields, saw 50\nSkipping line 89692: expected 49 fields, saw 50\nSkipping line 89693: expected 49 fields, saw 50\nSkipping line 89694: expected 49 fields, saw 50\nSkipping line 89695: expected 49 fields, saw 50\nSkipping line 89696: expected 49 fields, saw 50\nSkipping line 89697: expected 49 fields, saw 50\nSkipping line 8969

,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,...,AF own %,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95259,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,1001299.0,Connor,Rozee,PA,3,37,...,0.0,NaN,NaN,NaN,5.0,1.0,2.0,63.0,74.0,247.0
95260,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,290527.0,Trent,McKenzie,PA,3,37,...,0.0,NaN,NaN,NaN,2.0,0.0,1.0,81.0,97.0,136.0
95261,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,280990.0,Steven,Motlop,PA,9,36,...,0.0,NaN,NaN,NaN,5.0,0.0,1.0,83.0,73.0,144.0
95262,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,999346.0,Boyd,Woodcock,PA,2,29,...,0.0,NaN,NaN,NaN,1.0,0.0,3.0,62.0,75.0,89.0


### Clean player data

In [9]:
df_fanfooty_player_raw = df_fanfooty_player_raw.loc[df_fanfooty_player_raw['SC'] != '-', :]
df_fanfooty_player_raw['SC'] = df_fanfooty_player_raw['SC'].astype('int64')

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Identify when players were injured during a match
Fanfooty has amazing "tags" that can be used to identify when a player has been injured during a match

If they have certain tags (e.g. concussed) and score below 80 supercoach points, they are judged as injured.

In [10]:
injured_tags = [
    'sore',
    'injured',
    'longterminjured',
    'concussed',
    'heart'
]

def get_injured_status(row):
#     if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags) and row['SC'] < 80:
    if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags):
        return True
    else:
        return False

df_fanfooty_player_raw['Injured'] = df_fanfooty_player_raw.apply(lambda row: get_injured_status(row), axis=1)
df_fanfooty_player_raw

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,...,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Injured
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95259,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,1001299.0,Connor,Rozee,PA,3,37,...,NaN,NaN,NaN,5.0,1.0,2.0,63.0,74.0,247.0,False
95260,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,290527.0,Trent,McKenzie,PA,3,37,...,NaN,NaN,NaN,2.0,0.0,1.0,81.0,97.0,136.0,False
95261,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,280990.0,Steven,Motlop,PA,9,36,...,NaN,NaN,NaN,5.0,0.0,1.0,83.0,73.0,144.0,False
95262,7454,http://live.fanfooty.com.au/game/matchcentre.h...,R18,2020,999346.0,Boyd,Woodcock,PA,2,29,...,NaN,NaN,NaN,1.0,0.0,3.0,62.0,75.0,89.0,False


## 3. Match results/fixture

### Get fixture data from Fanfooty website

In [11]:
url = "http://www.fanfooty.com.au/resource/draw.php"
headers = ["FanFooty draw ID", "year", "competition", "round", "gametime (AET)", "day", "home team", "away team", "ground", "timeslot", "TV coverage", "home supergoals", "home goals", "home behinds", "home points", "away supergoals", "away goals", "away behinds", "away points", "match status"]
data_list = []


res = requests.get(url)
if res.text == 'Too many connections':
    raise
        
split_rows_list = res.text.split('"\r\n')
for row in split_rows_list:
    field_list = row.split(',')
    data_list.append(field_list)
    
df_fixture = pd.DataFrame(data_list, columns=headers)

for i, col in enumerate(df_fixture.columns):
    df_fixture.iloc[:, i] = df_fixture.iloc[:, i].str.replace('"', '')
    
df_fixture['FanFooty draw ID'] = pd.to_numeric(df_fixture['FanFooty draw ID'])
df_fixture

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,TV coverage,home supergoals,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status
0,1006.0,1993,HA,1,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,,,13,17,95,,17,13,115,Full Time
1,1000.0,1993,HA,1,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,,,24,22,166,,22,11,143,Full Time
2,1001.0,1993,HA,1,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,,,17,10,112,,17,16,118,Full Time
3,1002.0,1993,HA,1,1993-03-27 14:00:00,Saturday,Hawthorn,Melbourne,Waverley,D,,,13,15,93,,11,4,70,Full Time
4,1005.0,1993,HA,1,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,,,20,16,136,,16,16,112,Full Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669,7460.0,2020,SF,20,2020-10-10 19:40:00,Saturday,Geelong,Collingwood,Gabba,N,,0,15,10,100,0,5,2,32,Full Time
5670,7461.0,2020,PF,21,2020-10-16 19:50:00,Friday,Port Adelaide,Richmond,Adelaide,N,,,,,,,,,,
5671,7462.0,2020,PF,21,2020-10-17 19:40:00,Saturday,Brisbane Lions,Geelong,Gabba,N,,,,,,,,,,
5672,7463.0,2020,GF,22,2020-10-24 19:50:00,Saturday,TBD,TBD,TBD,N,,,,,,,,,,


# Get standard team name

In [12]:
df_team_names = pd.read_csv("inputs/all_sources_team_names.csv")
df_fixture = df_fixture.merge(df_team_names[['fanfooty_team_name', 'fanfooty_team_name_mid']], left_on='home team' ,right_on='fanfooty_team_name_mid', how='left')
df_fixture = df_fixture.rename(columns={"fanfooty_team_name": "home_team_short"})
del df_fixture['fanfooty_team_name_mid']
df_fixture = df_fixture.merge(df_team_names[['fanfooty_team_name', 'fanfooty_team_name_mid']], left_on='away team' ,right_on='fanfooty_team_name_mid', how='left')
df_fixture = df_fixture.rename(columns={"fanfooty_team_name": "away_team_short"})
del df_fixture['fanfooty_team_name_mid']
df_fixture.drop(df_fixture.tail(1).index,inplace=True)
df_fixture

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,...,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status,home_team_short,away_team_short
0,1006.0,1993,HA,1,1993-03-26 20:08:00,Friday,Western Bulldogs,Collingwood,MCG,N,...,13,17,95,,17,13,115,Full Time,WB,CO
1,1000.0,1993,HA,1,1993-03-27 14:00:00,Saturday,North Melbourne,Brisbane Bears,MCG,D,...,24,22,166,,22,11,143,Full Time,NM,NaN
2,1001.0,1993,HA,1,1993-03-27 14:00:00,Saturday,Carlton,Fitzroy,Princes Park,D,...,17,10,112,,17,16,118,Full Time,CA,NaN
3,1002.0,1993,HA,1,1993-03-27 14:00:00,Saturday,Hawthorn,Melbourne,Waverley,D,...,13,15,93,,11,4,70,Full Time,HW,ME
4,1005.0,1993,HA,1,1993-03-27 14:08:00,Saturday,Geelong,St Kilda,Kardinia,D,...,20,16,136,,16,16,112,Full Time,GE,SK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5668,7459.0,2020,SF,20,2020-10-09 19:50:00,Friday,Richmond,St Kilda,Gold Coast,N,...,12,8,80,0,6,13,49,Full Time,RI,SK
5669,7460.0,2020,SF,20,2020-10-10 19:40:00,Saturday,Geelong,Collingwood,Gabba,N,...,15,10,100,0,5,2,32,Full Time,GE,CO
5670,7461.0,2020,PF,21,2020-10-16 19:50:00,Friday,Port Adelaide,Richmond,Adelaide,N,...,,,,,,,,,PA,RI
5671,7462.0,2020,PF,21,2020-10-17 19:40:00,Saturday,Brisbane Lions,Geelong,Gabba,N,...,,,,,,,,,BL,GE


### Get the total SuperCoach and AFL Fantasy scores for each team, for every match

In [13]:
# Create a summary to get the total SC and AF points for each match
df_match_summary = pd.pivot_table(df_fanfooty_player_raw, index=['Fanfooty Match ID'], values=['SC'], columns=['Team'], aggfunc=np.sum)
df_match_summary = df_match_summary.reset_index()
headings = [x[1] for x in df_match_summary.columns]
headings[0] = 'Fanfooty Match ID'
df_match_summary.columns = headings
df_match_summary

,Fanfooty Match ID,AD,BL,CA,CO,ES,FR,GC,GE,HW,ME,NM,PA,RI,SK,SY,WB,WC,WS
0,3425,NaN,NaN,NaN,NaN,1568.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1739.0,NaN
1,3426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1504.0,NaN,NaN,NaN,1797.0,NaN,NaN,NaN
2,3427,1513.0,NaN,1826.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3428,NaN,NaN,NaN,1873.0,NaN,NaN,NaN,NaN,1447.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3429,NaN,1781.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1545.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,7450,NaN,1749.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2158,7451,NaN,NaN,NaN,NaN,NaN,NaN,1383.0,NaN,1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2159,7452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1660.0,NaN,NaN,NaN,NaN,NaN,NaN,1640.0,NaN,NaN,NaN
2160,7453,NaN,NaN,NaN,NaN,NaN,1541.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1761.0,NaN,NaN


### Join each team's SC total to fixture data + save file

In [15]:
def get_sc_total(row, team_short):
    match_id = row['FanFooty draw ID']
    team_name = row[team_short]
    if match_id >= 3425:
        sc_total = df_match_summary.loc[df_match_summary['Fanfooty Match ID'] == match_id, team_name]
        if len(sc_total) > 0: 
            return sc_total.values[0]

df_fixture['Home SC total'] = df_fixture.apply(lambda row: get_sc_total(row, 'home_team_short'), axis=1)
df_fixture['Away SC total'] = df_fixture.apply(lambda row: get_sc_total(row, 'away_team_short'), axis=1)
df_fixture.to_csv('{}/fanfooty_fixture.csv'.format(destination))
df_fixture

KeyError: nan

### Add home/away + opposition columns to player data

In [ ]:
def get_opposition(row):
    match_id = row['Fanfooty Match ID']
    team = row['Team']
    home_team = df_fixture.loc[(df_fixture['FanFooty draw ID'] == match_id) & (df_fixture['home_team_short'] == team)]
    away_team = df_fixture.loc[(df_fixture['FanFooty draw ID'] == match_id) & (df_fixture['away_team_short'] == team)]
    if len(home_team) > 0:
        return home_team['away_team_short'].values[0]
    if len(away_team) > 0:
        return away_team['home_team_short'].values[0]
    

df_fanfooty_player_raw['Opposition'] = df_fanfooty_player_raw.apply(lambda row: get_opposition(row), axis=1)
df_fanfooty_player_raw.to_csv("{}/{}".format(destination, file_name))
df_fanfooty_player_raw

In [ ]:
"{}/{}".format(destination, file_name)